In [150]:
import pandas as pd;
import re

df = pd.read_excel('../Processadores.xlsx')

In [151]:
df['preco_anterior'] = df['preco_anterior'].str.replace('R$', '')
df['preco_anterior'] = df['preco_anterior'].str.replace('.', '')
df['preco_anterior'] = df['preco_anterior'].str.replace(',', '.')
df['preco_anterior'] = df['preco_anterior'].str.strip()
print(df['preco_anterior']) 

0      2222.21
1          NaN
2      1222.21
3      1326.22
4      1068.17
        ...   
369        NaN
370        NaN
371        NaN
372        NaN
373        NaN
Name: preco_anterior, Length: 374, dtype: object


In [152]:
df['preco_anterior'] = df['preco_anterior'].astype(float)
print(df['preco_anterior'])

0      2222.21
1          NaN
2      1222.21
3      1326.22
4      1068.17
        ...   
369        NaN
370        NaN
371        NaN
372        NaN
373        NaN
Name: preco_anterior, Length: 374, dtype: float64


In [153]:
def formatar_brl_sem_cifrao(valor):
    if pd.isna(valor):
        return ''
    return '{:,.2f}'.format(valor).replace(',', 'X').replace('.', ',').replace('X', '.')

df['preco_anterior'] = df['preco_anterior'].apply(formatar_brl_sem_cifrao)
print(df['preco_anterior'])

0      2.222,21
1              
2      1.222,21
3      1.326,22
4      1.068,17
         ...   
369            
370            
371            
372            
373            
Name: preco_anterior, Length: 374, dtype: object


In [154]:
df['preco_atual'] = df['preco_atual'].str.replace('R$', '')
df['preco_atual'] = df['preco_atual'].str.replace('.', '')
df['preco_atual'] = df['preco_atual'].str.replace(',', '.')
df['preco_atual'] = df['preco_atual'].str.strip()
print(df['preco_atual'])

0      1749.99
1       656.01
2       999.99
3       599.99
4       749.99
        ...   
369    1664.99
370     399.90
371    3160.23
372     116.04
373    2635.38
Name: preco_atual, Length: 374, dtype: object


In [155]:
df['preco_atual'] = df['preco_atual'].astype(float)
print(df['preco_atual'][0])

1749.99


In [156]:
def formatar_brl_sem_cifrao(valor):
    if pd.isna(valor):
        return ''
    return '{:,.2f}'.format(valor).replace(',', 'X').replace('.', ',').replace('X', '.')

df['preco_atual'] = df['preco_atual'].apply(formatar_brl_sem_cifrao)
print(df['preco_atual'])

0      1.749,99
1        656,01
2        999,99
3        599,99
4        749,99
         ...   
369    1.664,99
370      399,90
371    3.160,23
372      116,04
373    2.635,38
Name: preco_atual, Length: 374, dtype: object


In [157]:
df['preco_anterior'].fillna('Desconhecido', inplace=True)
print(df['preco_anterior'])

0      2.222,21
1              
2      1.222,21
3      1.326,22
4      1.068,17
         ...   
369            
370            
371            
372            
373            
Name: preco_anterior, Length: 374, dtype: object


C:\Users\48260449846\AppData\Local\Temp\ipykernel_8992\350384653.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['preco_anterior'].fillna('Desconhecido', inplace=True)


In [158]:
df = df.dropna()
print(len(df))

70


In [159]:
# Extrações do título
MARCAS = {
    'AMD': 'AMD',
    'Intel': 'Intel',
    'Amd': 'Amd'
}

def extrair_info(titulo):
    marca = next((v for k, v in MARCAS.items() if k in titulo), 'Desconhecida')
    modelo_match = re.search(r'(Ryzen \d+ \S+|Core i\d+-\d+\S*|Core I\d+-\S|Core I\d+ - \S*|Core I\d+ \S*|Core \d+Ultra\S*|Ultra \d+\S*|Celeron \d+ \S*|Pentium \d+ Gold \S*|Ryzen \d+Threadripper Pro\S*|Ryzen \d+ Radeon \S)', titulo)
    modelo = modelo_match.group(0) if modelo_match else 'Desconhecido'
    freq_base = re.search(r'(\d+(\.\d+)?)\s*GHz', titulo)
    freq_turbo = re.search(r'\((\d+(\.\d+)?)GHz', titulo)
    cache_match = re.search(r'Cache (\d+)MB', titulo)

    return pd.Series({
        'marca': marca,
        'modelo': modelo,
        'freq_base_GHz': float(freq_base.group(1)) if freq_base else None,
        'freq_turbo_GHz': float(freq_turbo.group(1)) if freq_turbo else None,
        'cache_MB': int(cache_match.group(1)) if cache_match else None
    })

info_extraida = df['titulo'].apply(extrair_info)
df = pd.concat([df, info_extraida], axis=1)
print(df)

                                                titulo preco_atual  \
6    Processador AMD Ryzen 7 5700X, 3.4GHz (4.6GHz ...    1.319,99   
11   Processador AMD Ryzen 5 9600X, 3.9 GHz (5.4 GH...    1.649,99   
18   Processador Intel Core i5-14600K, 14ª Geração,...    1.749,99   
34   Processador Intel Core i7-12700F, 2.1GHz (4.9G...    1.699,99   
35   Processador AMD Ryzen 7 8700G, 4.2GHz (5.1GHz ...    1.949,99   
..                                                 ...         ...   
226  Processador AMD Ryzen Threadripper Pro 5995WX,...   37.999,99   
236  Processador Intel Core I9-12900KF, 3.2GHz, Cac...    2.979,00   
249  Processador Intel Pentium Gold G6405, 10ª Gera...      509,00   
289  Processador Intel Core Ultra 7 265kf Arrow Lak...    2.669,00   
331  Processador Intel Ultra 5 245k Arrow Lake 4.2g...    2.469,00   

    preco_anterior       em_estoque  marca           modelo  freq_base_GHz  \
6         2.840,90  Restam 80 unid.    AMD   Ryzen 7 5700X,            3.4   
11 

In [160]:
# df = df.dropna()
# print(len(df))

In [161]:
df.to_excel('processadores_filtrados.xlsx', index=False)